In [2]:
import os
import pandas as pd 
import numpy as np

In [117]:
### READ IN OUR DATA #### 
# read in od deaths data
od = pd.read_csv('od_deaths_fips.csv')
od = od.drop('Unnamed: 0', axis = 1)

# change FIPS to 5 letter string
new_col = od['FIPS'].astype(int).astype(str)
od['FIPS'] = new_col.apply(lambda x: x.zfill(5))
od 

# read in adjacency data
spatcomp = pd.read_csv('county_adjacency.csv')

# Add a leading zero to our fips code 
new_col = spatcomp['fipscounty'].astype(int).astype(str)
spatcomp['fipscounty'] = new_col.apply(lambda x: x.zfill(5))

new_col = spatcomp['fipsneighbor'].astype(int).astype(str)
spatcomp['fipsneighbor'] = new_col.apply(lambda x: x.zfill(5))

# removing rows where the adjacent county is itself
spatcomp = spatcomp[spatcomp['fipscounty'] != spatcomp['fipsneighbor']]
spatcomp.columns = ['countyname', 'fipscounty', 'neighborname', 'FIPS']
spatcomp

,countyname,fipscounty,neighborname,FIPS
1,"Autauga County, AL",01001,"Chilton County, AL",01021
2,"Autauga County, AL",01001,"Dallas County, AL",01047
3,"Autauga County, AL",01001,"Elmore County, AL",01051
4,"Autauga County, AL",01001,"Lowndes County, AL",01085
5,"Autauga County, AL",01001,"Montgomery County, AL",01101
...,...,...,...,...
22191,"Yauco Municipio, PR",72153,"Lares Municipio, PR",72081
22192,"Yauco Municipio, PR",72153,"Maricao Municipio, PR",72093
22193,"Yauco Municipio, PR",72153,"Sabana Grande Municipio, PR",72121
22197,"St. John Island, VI",78020,"St. Thomas Island, VI",78030


In [119]:

spatialcomp = pd.DataFrame()

for i in range(2010, 2021): 
    
    # filter overdose df to current year 
    od10 = od[od['Year'] == i]
    # get only necessary columns 
    od10_rates = od10[['FIPS', 'Cruder Rate']]
    
    # adding od rates to spatial df according to the neighbor county fips code 
    merged = spatcomp.merge(od10_rates, on = ['FIPS'], how = 'left') 
    
    # get the mean od rates of the neighboring counties for each of the focal county 
    spatmean10 = merged.groupby('fipscounty').mean()
    # turn our groupby series into a df 
    spatmean10['fipscounty'] = spatmean10.index
    spatmean10 = spatmean10.reset_index(drop=True)
    
    # do same for max od rates 
    spatmax10 = merged.groupby('fipscounty')['Cruder Rate'].max()
    spatmax10 = pd.DataFrame(spatmax10)
    spatmax10['fipscounty'] = spatmax10.index
    spatmax10 = spatmax10.reset_index(drop=True)
    
    # organizing spatial df into nice form 
    spatmax10.columns = ['spatmax', 'fipscounty']
    spatmax10['Year'] = i
    spatcomp10 = spatmax10
    spatcomp10['spatmean'] = spatmean10['Cruder Rate']
    spatcomp10 = spatcomp10[['Year', 'fipscounty', 'spatmax', 'spatmean']] 
    
    # append each year's spatial df to the ones we already made 
    spatialcomp = pd.concat([spatialcomp, spatcomp10]).reset_index(drop=True)
    
print(spatialcomp)

       Year fipscounty    spatmax   spatmean
0      2010      01001  25.204500  16.526161
1      2010      01003  26.096714  16.138937
2      2010      01005        NaN        NaN
3      2010      01007  25.204500  15.378684
4      2010      01009  50.728854  22.318995
...     ...        ...        ...        ...
35459  2020      72149        NaN        NaN
35460  2020      72151        NaN        NaN
35461  2020      72153        NaN        NaN
35462  2020      78020        NaN        NaN
35463  2020      78030        NaN        NaN

[35464 rows x 4 columns]


In [142]:
# spatialcomp.to_csv('spatialcomp.csv') 

In [42]:
merged.nunique()
# Some Hawaiian counties are islands, duh  

countyname      3222
fipscounty      3224
neighborname    3230
FIPS            3224
Cruder Rate      776
dtype: int64